<h1>Máster de Ciencia de Datos e Ingenieria de computadores.</h1>
<h2> Visión por Computador </h2>
<h3> <strong> Autores: </strong>Brian Sena Simons y Miguel García López </h3>

# Inúndación Texas

# Arizona

# Adra

# Bibliografía

[1]
[2]
[3]